In [39]:
from utils import data_process,load_data
import os
# from bulk2space import Bulk2Space
import scanpy
from vae import train_vae,BulkDataset
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader,Dataset
used_device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
input_dir=r"../tutorial/data/example_data/demo1/"

In [3]:
input_sc_data_path=os.path.join(input_dir,"demo1_sc_data.csv")
input_bulk_path=os.path.join(input_dir,"demo1_bulk.csv")
input_sc_meta_path=os.path.join(input_dir,"demo1_sc_meta.csv")
input_st_data_path=os.path.join(input_dir,"demo1_st_data.csv")
input_st_meta_path=os.path.join(input_dir,"demo1_st_meta.csv")

In [4]:
input_data=load_data(input_bulk_path,input_sc_data_path,input_sc_meta_path,
                     input_st_data_path,input_st_meta_path)

loading data......
load data done!


In [5]:
input_data.keys()

dict_keys(['input_sc_meta', 'sc_gene', 'bulk_gene', 'intersect_gene', 'input_sc_data', 'input_bulk', 'input_st_meta', 'input_st_data'])

In [6]:
input_data["input_sc_data"]

,C_417,C_369,C_265,C_450,C_440,C_347,C_463,C_429,C_442,C_430,...,C_269,C_257,C_50,C_304,C_313,C_439,C_363,C_90,C_54,C_289
TMEM219,0.0,0.000000,1.43938,0.000000,0.0,0.000000,0.0,0.000000,2.010303,0.0,...,0.000000,2.1534,1.554921,0.000000,0.000000,0.000000,0.000000,0.000000,1.677299,0.0
CCL5,0.0,0.000000,1.43938,0.000000,0.0,0.000000,0.0,0.000000,2.659744,0.0,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
TBC1D14,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.0000,0.000000,0.000000,1.923612,1.775942,0.000000,0.000000,0.000000,0.0
DDRGK1,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.0000,0.000000,2.113417,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
ACTG2,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,1.534554,0.0,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,1.508931,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ATP5PB,0.0,0.000000,0.00000,1.948529,0.0,0.000000,0.0,1.312637,0.000000,0.0,...,0.000000,0.0000,0.000000,0.000000,0.000000,1.775942,1.508931,1.571530,0.000000,0.0
LMAN1,0.0,2.185951,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,2.198419,0.0000,0.000000,2.113417,0.000000,0.000000,0.000000,0.000000,2.161002,0.0
BAD,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,1.312637,0.000000,0.0,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,2.054519,0.000000,0.0
FDFT1,0.0,0.000000,0.00000,0.000000,0.0,1.389041,0.0,0.000000,0.000000,0.0,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.677299,0.0


In [7]:
processed_data=data_process(input_data,500,1)

c:\ProgramData\Anaconda3\lib\site-packages\anndata\_core\anndata.py:1222: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = c
c:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


In [8]:
processed_data

{'B cell': 45.0, 'Dendritic cell': 25.0, 'Monocyte': 103.0, 'T cell': 76.0}

In [9]:
def __get_model_input(data, cell_target_num):
    # input：data， celltype， bulk & output: label, dic, single_cell
    single_cell = data["input_sc_data"].values.T  # single cell data (600 * 6588)
    index_2_gene = (data["input_sc_data"].index).tolist()
    breed = data["input_sc_meta"]['Cell_type']
    breed_np = breed.values
    breed_set = set(breed_np)
    breed_2_list = list(breed_set)
    dic = {}  # breed_set to index {'B cell': 0, 'Monocyte': 1, 'Dendritic cell': 2, 'T cell': 3}
    label = []  # the label of cell (with index correspond)
    nclass = len(breed_set)

    ntrain = single_cell.shape[0]
    # FeaSize = single_cell.shape[1]
    feature_size = single_cell.shape[1]
    assert nclass == len(cell_target_num.keys()), "cell type num no match!!!"

    for i in range(len(breed_set)):
        dic[breed_2_list[i]] = i
    cell = data["input_sc_meta"]["Cell"].values

    for i in range(cell.shape[0]):
        label.append(dic[breed_np[i]])

    label = np.array(label)

    # label index the data size of corresponding target
    cell_number_target_num = {}
    for k, v in cell_target_num.items():
        cell_number_target_num[dic[k]] = v

    return single_cell, label, breed_2_list, index_2_gene, cell_number_target_num, nclass, ntrain, feature_size



In [10]:
# Bs=Bulk2Space()
# Bs.__get_model_input()
single_cell, label, breed_2_list, index_2_gene, cell_number_target_num, \
        nclass, ntrain, feature_size = __get_model_input(input_data, processed_data)


In [34]:
# single_cell.shape
# pd.DataFrame(data=single_cell).T
# breed_2_list
# index_2_gene
# nclass
import random
random.sample(range(10),5)

[2, 5, 7, 1, 8]

In [11]:
print('begin vae training...')
vae_net = train_vae(single_cell,
                    label,
                    used_device,
                    feature_size=feature_size,
                    batch_size=512,learning_rate=1e-4,
    hidden_size=256,epoch_num=20)
print('vae training done!')


begin vae training...


  0%|          | 0/20 [00:00<?, ?it/s]h:\Bulk2space\bulk2space\vae.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cell_feature = torch.tensor(cell_feature, dtype=torch.float32).to(used_device)
Train Epoch: 19: 100%|██████████| 20/20 [00:09<00:00,  2.03it/s, loss=0.5186, min_loss=0.5186]

min loss = 0.5185707807540894
vae training done!


In [12]:
next(vae_net.parameters()).device

device(type='cpu')

In [13]:
def prepare_data(cell_all_generate, label_all_generate, breed_2_list, index_2_gene):
    cell_all_generate = np.array(cell_all_generate)
    label_all_generate = np.array(label_all_generate)

    cell_all_generate_csv = pd.DataFrame(cell_all_generate)
    label_all_generate_csv = pd.DataFrame(label_all_generate)

    ids = label_all_generate_csv[0].tolist()
    breeds = []
    for id in ids:
        breeds.append(breed_2_list[id])
    name = ["C_" + str(i + 1) for i in range(label_all_generate.shape[0])]

    label_all_generate_csv.insert(1, "Cell_type", np.array(breeds))
    label_all_generate_csv.insert(1, "Cell", np.array(name))
    label_all_generate_csv = label_all_generate_csv.drop([0], axis=1)

    cell_all_generate_csv = cell_all_generate_csv.T
    cell_all_generate_csv.columns = name
    cell_all_generate_csv.index = index_2_gene

    return label_all_generate_csv, cell_all_generate_csv


In [14]:
def generate_vae(net, ratio, single_cell, label, breed_2_list, index_2_gene, cell_number_target_num=None,
                 used_device=None):
    '''
    使用初步训练好的vae进行单细胞生成
    '''
    # net in cuda now
    for p in net.parameters():  # reset requires_grad
        p.requires_grad = False  # avoid computation

    net.eval()
    net.to(used_device)
    cell_all_generate = []
    label_all_generate = []

    all_to_generate = 0
    for x in cell_number_target_num.values():
        all_to_generate += x

    if cell_number_target_num != None:
        epochs = 10000 
        ratio = 1
    else:
        epochs = 1

    cell_feature = torch.from_numpy(single_cell).float()
    label = torch.from_numpy(label)

    with torch.no_grad():
        with tqdm(total=all_to_generate, desc='generating') as pbar:
            for epoch in range(epochs):
                key_list = []  # list
                generate_num = 0

                label_list = label.tolist()
                for i in range(len(label_list)):
                    if cell_number_target_num[label_list[i]] <= 0:
                        continue
                    else:
                        cell_number_target_num[label_list[i]] -= 1
                        generate_num += 1
                        key_list.append(i)

                if cell_number_target_num == None or all_to_generate == 0 or len(key_list) == 0:
                    assert all_to_generate == 0 and len(key_list) == 0
                    break

                import random
                random.shuffle(key_list)

                label = label.index_select(0, torch.tensor(key_list))
                cell_feature = cell_feature.index_select(0, torch.tensor(key_list))

                dataloader = DataLoader(BulkDataset(single_cell=cell_feature, label=label), batch_size=300,
                                        shuffle=False,
                                        pin_memory=True, num_workers=0)
                for batch_idx, data in enumerate(dataloader):  
                    cell_feature_batch, label_batch = data
                    cell_feature_batch = cell_feature_batch.to(used_device)

                    label_batch = label_batch.cpu().numpy()

                    for j in range(ratio): 
                        ans_l, _ = net(cell_feature_batch, used_device)
                        ans_l = ans_l.cpu().data.numpy()
                        cell_all_generate.extend(ans_l)
                        label_all_generate.extend(label_batch)

                all_to_generate -= generate_num
                pbar.update(generate_num)

    print("generated done!")
    generate_sc_meta, generate_sc_data = prepare_data(cell_all_generate, label_all_generate, breed_2_list,
                                                      index_2_gene)
    return generate_sc_meta, generate_sc_data



In [15]:
generate_sc_meta, generate_sc_data = generate_vae(vae_net, -1,
                                                          single_cell, label, breed_2_list,
                                                          index_2_gene, cell_number_target_num, used_device)


generating: 100%|██████████| 249/249.0 [37:06<00:00,  8.94s/it]

generated done!


In [24]:
single_cell.shape

(250, 6588)

In [27]:
# generate_sc_meta["Cell"].value_counts()
generate_sc_meta["Cell_type"].value_counts()

Cell_type
Monocyte          103
T cell             76
B cell             45
Dendritic cell     25
Name: count, dtype: int64

In [43]:
# generate_sc_meta.Cell
# sorted(list(set(generate_sc_meta.Cell)))
pd.__version__


'2.2.0'

In [36]:
input_st_data = pd.read_csv(os.path.join(input_dir,"demo1_st_data.csv"), index_col=0)
input_st_meta = pd.read_csv(os.path.join(input_dir,"demo1_st_meta.csv"), index_col=0)


In [44]:
def create_st(generate_sc_data, generate_sc_meta, spot_num, cell_num, gene_num, marker_used):
    sc = generate_sc_data
    sc_ct = generate_sc_meta
    cell_name = sorted(list(set(sc_ct.Cell)))

    last_cell_pool = []
    spots = pd.DataFrame()
    meta = pd.DataFrame(columns=['Cell', 'Celltype', 'Spot'])
    sc_ct.index = sc_ct['Cell']
    for i in range(spot_num):
        cell_pool = random.sample(cell_name, cell_num)     ##从cell_name中随机挑选cell_num个cell
        while set(cell_pool) == set(last_cell_pool):
            cell_pool = random.sample(cell_name, cell_num)
        last_cell_pool = cell_pool
        syn_spot = sc[cell_pool].sum(axis=1)
        if syn_spot.sum() > 25000:
            syn_spot *= 20000 / syn_spot.sum()
        spot_name = f'spot_{i + 1}'
        spots.insert(len(spots.columns), spot_name, syn_spot)   ##在dataframe后插入spot_name列，值为synspot

        for cell in cell_pool:
            celltype = sc_ct.loc[cell, 'Cell_type']
            row = {'Cell': cell, 'Celltype': celltype, 'Spot': spot_name}
            # meta = meta.append(row, ignore_index=True)
            meta=pd.concat([meta,pd.DataFrame([row])],ignore_index=True)

    if marker_used:
        adata = scanpy.AnnData(sc.T)

        adata.obs = sc_ct[['Cell_type']]
        scanpy.tl.rank_genes_groups(adata, 'Cell_type', method='wilcoxon')
        marker_df = pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(gene_num)
        marker_array = np.array(marker_df)
        marker_array = np.ravel(marker_array)       ##将数据拉平为一维
        marker_array = np.unique(marker_array)
        marker = list(marker_array)
        sc = sc.loc[marker, :]
        spots = spots.loc[marker, :]
        
    return sc, sc_ct, spots, meta



In [ ]:
sc,sc_ct,spots,meta=create_st(generate_sc_data=generate_sc_data,generate_sc_meta=generate_sc_meta,
                              spot_num=500,cell_num=10,gene_num=500,marker_used=True)

In [54]:
def create_sample(sc, st, meta, multiple):
    cell_name = meta.Cell.values.tolist()
    spot_name = meta.Spot.values.tolist()

    # get wrong spot name for negative data
    all_spot = list(set(meta.Spot))
    wrong_spot_name = []
    for sn in spot_name:
        last_spot = all_spot.copy()  # --
        last_spot.remove(sn)  # --
        mul_wrong = random.sample(last_spot, multiple)
        wrong_spot_name.extend(mul_wrong)

    cfeat_p_list, cfeat_n_list = [], []

    for c in cell_name:
        cell_feat = sc[c].values.tolist()

        cfeat_p_list.append(cell_feat)
        cfeat_m = [cell_feat * multiple]
        cfeat_n_list.extend(cfeat_m)

    cfeat_p = np.array(cfeat_p_list)  # [n, d]
    cfeat_n = np.array(cfeat_n_list).reshape(-1, cfeat_p.shape[1])  # [n*m, d]

    # positive spot features
    sfeat_p_list = []
    for s in spot_name:
        spot_feat = st[s].values.tolist()
        sfeat_p_list.append(spot_feat)
        # sfeat_p = np.vstack((sfeat_p, spot_feat))
    sfeat_p = np.array(sfeat_p_list)  # [n, d]

    mfeat_p = sfeat_p - cfeat_p
    feat_p = np.hstack((cfeat_p, sfeat_p))
    feat_p = np.hstack((feat_p, mfeat_p))
    print('sucessfully create positive data')

    # negative spot features
    sfeat_n_list = []
    for s in wrong_spot_name:
        spot_feat = st[s].values.tolist()
        sfeat_n = sfeat_n_list.append(spot_feat)
    sfeat_n = np.array(sfeat_n_list)

    mfeat_n = sfeat_n - cfeat_n
    feat_n = np.hstack((cfeat_n, sfeat_n))
    feat_n = np.hstack((feat_n, mfeat_n))
    print('sucessfully create negative data')

    return feat_p, feat_n



In [56]:
pos_train, neg_train = create_sample(sc, spots, meta, multiple=1)

sucessfully create positive data
sucessfully create negative data


In [53]:
neg_train.shape

(5000, 5724)

In [57]:
def get_data(pos, neg):
    X = np.vstack((pos, neg))
    y = np.concatenate((np.ones(pos.shape[0]), np.zeros(neg.shape[0])))

    return X, y



In [58]:
x_train,y_train=get_data(pos_train,neg_train)

In [63]:
# 1908*3==x_train.shape[1]
os.cpu_count()

8

In [62]:
spots

,spot_1,spot_2,spot_3,spot_4,spot_5,spot_6,spot_7,spot_8,spot_9,spot_10,...,spot_491,spot_492,spot_493,spot_494,spot_495,spot_496,spot_497,spot_498,spot_499,spot_500
ABCF1,4.745543,4.822631,4.568675,4.634288,4.658358,4.917338,5.050748,4.910650,4.403915,4.605649,...,4.608803,4.746439,4.877976,4.848539,5.211403,4.835359,4.859098,4.848010,4.613491,4.938049
ABLIM1,4.991643,4.891234,4.518946,5.005117,4.787114,5.205488,4.888621,5.164276,4.543599,4.763700,...,4.978578,4.967765,4.978430,4.859894,4.872295,4.695004,4.980745,4.759206,4.646165,5.118901
ABR,1.871212,2.249499,2.020125,2.025755,1.919767,2.269598,2.247834,1.876305,1.992937,1.895729,...,2.186847,2.269282,2.025875,2.031499,2.311798,1.665364,2.167269,1.954075,2.082141,2.039951
ABRACL,8.182046,8.422922,8.361147,8.169659,8.526273,8.727376,8.474186,8.759098,7.503559,7.779015,...,8.656209,8.076756,8.433110,8.034408,8.796514,8.418385,8.686009,8.141794,8.006170,8.463879
ACAA1,1.948504,2.218957,2.014586,1.830430,2.227700,2.251006,2.263686,2.089791,1.925685,1.777642,...,2.121410,1.928137,1.805997,1.865915,2.377412,1.887871,1.745851,2.060977,2.044411,2.252948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZRANB1,0.483906,0.402116,0.372007,0.567658,0.593144,0.444906,0.493170,0.485752,0.526892,0.457979,...,0.449356,0.399427,0.542594,0.286482,0.308857,0.450071,0.486942,0.456927,0.527298,0.412845
ZRSR2,1.963209,2.505477,2.308692,2.364383,2.477764,2.593113,2.203200,2.367003,2.147051,2.188067,...,2.306390,2.303512,2.320311,2.218030,2.570249,2.102299,2.348050,2.002709,2.464188,2.376592
ZYX,6.708426,6.914726,6.508217,6.580233,7.062467,7.394115,7.067342,6.968837,6.410253,6.335495,...,6.850892,6.677891,6.922782,6.529444,7.368014,6.564146,6.886856,6.702064,6.817007,6.983569
ZZEF1,0.239493,0.291918,0.172057,0.050620,0.043357,0.324694,0.238431,0.252300,0.148464,0.228573,...,0.275307,0.046429,0.327636,0.406256,0.232774,0.158644,0.071530,0.322480,0.154350,0.142808


In [38]:
def create_data(generate_sc_meta, generate_sc_data, st_data, spot_num, cell_num, top_marker_num, marker_used,
                mul_train):
    ##由此创造深度森林的训练数据
    sc_gene = generate_sc_data._stat_axis.values.tolist()
    st_gene = st_data._stat_axis.values.tolist()
    intersect_gene = list(set(sc_gene).intersection(set(st_gene)))
    generate_sc_data = generate_sc_data.loc[intersect_gene]
    st_data = st_data.loc[intersect_gene]
    sc_train, _, st_train, meta_train = create_st(generate_sc_data, generate_sc_meta,
                                                  spot_num, cell_num,
                                                  top_marker_num, marker_used)
    pos_train, neg_train = create_sample(sc_train, st_train, meta_train, mul_train)
    xtrain, ytrain = get_data(pos_train, neg_train)
    
    return xtrain, ytrain



Index(['spot_1', 'spot_2', 'spot_3', 'spot_4', 'spot_5', 'spot_6', 'spot_7',
       'spot_8', 'spot_9', 'spot_10', 'spot_11', 'spot_12', 'spot_13',
       'spot_14', 'spot_15', 'spot_16', 'spot_17', 'spot_18', 'spot_19',
       'spot_20', 'spot_21', 'spot_22', 'spot_23', 'spot_24', 'spot_25',
       'spot_26', 'spot_27', 'spot_28', 'spot_29', 'spot_30'],
      dtype='object')